<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score

In [2]:
df = pd.DataFrame({'x1': [0, 0, 1, 0, 1, 1, 0],
                   'x2': [0, 1, 0, 1, 0, 1, 0],
                   'x3': [1, 1, 1, 0, 0, 1, 0],
                   'y':  [0, 1, 1, 1, 1, 0, 0]})

In [3]:
df

,x1,x2,x3,y
0,0,0,1,0
1,0,1,1,1
2,1,0,1,1
3,0,1,0,1
4,1,0,0,1
5,1,1,1,0
6,0,0,0,0


In [4]:
feat = ['x1', 'x2', 'x3']
X = df[feat].values
y = np.expand_dims(df['y'], axis=1)

In [5]:
class NeuralNetwork:
    def __init__(self, inputs=2, hiddenNodes=3, outputNodes=1):
        # Set up Architecture of Neural Network
        self.inputs = inputs
        self.hiddenNodes = hiddenNodes
        self.outputNodes = outputNodes

        # Initial Weights
        # Inputs x hidden nodes Matrix Array for the First Layer
        self.weights1 = np.random.rand(self.inputs, self.hiddenNodes)
       
        # Hidden nodes x outputs Matrix Array for Hidden to Output
        self.weights2 = np.random.rand(self.hiddenNodes, self.outputNodes)
        
    def sigmoid(self, s):
        return 1 / (1 + np.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1 - s)
    
    def feed_forward(self, X):
        """
        Calculate the NN inference using feed forward.
        aka "predict"
        """
        
        # Weighted sum of inputs => hidden layer
        self.hidden_sum = np.dot(X, self.weights1)
        
        # Activations of weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        # Weight sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        # Final activation of output
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output
        
    def backward(self, X, y, o):
        """
        Backward propagate through the network
        """
        
        # Error in Output
        self.o_error = y - o
        
        # Apply Derivative of Sigmoid to error
        # How far off are we in relation to the Sigmoid f(x) of the output
        # ^- aka hidden => output
        self.o_delta = self.o_error * self.sigmoidPrime(o)
        
        # z2 error
        self.z2_error = self.o_delta.dot(self.weights2.T)
        
        # How much of that "far off" can explained by the input => hidden
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.activated_hidden)
        
        # Adjustment to first set of weights (input => hidden)
        self.weights1 += X.T.dot(self.z2_delta)
        # Adjustment to second set of weights (hidden => output)
        self.weights2 += self.activated_hidden.T.dot(self.o_delta)
        

    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X, y, o)

    def fit(self, X, y, max_epochs, print_epochs):
        for i in range(max_epochs):
            if (i+ 1 in print_epochs): 
                print('+' + '---' * 3 + f'EPOCH {i + 1}' + '---' * 3 + '+')
                print('Input: \n', X)
                print('Actual Output: \n', y)
                print('Predicted Output: \n', str(self.feed_forward(X)))
                print("Loss: \n", str(np.mean(np.square(y - self.feed_forward(X)))))
            self.train(X, y)

In [6]:
nn = NeuralNetwork(inputs=3, hiddenNodes=4, outputNodes=1)

In [7]:
nn.fit(X, y, 6000, [1, 2, 3000, 6000])

+---------EPOCH 1---------+
Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.74514296]
 [0.79040335]
 [0.79620246]
 [0.77148492]
 [0.77842465]
 [0.82487661]
 [0.71917953]]
Loss: 
 0.2770939446433033
+---------EPOCH 2---------+
Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.67201853]
 [0.70776606]
 [0.71484477]
 [0.6935456 ]
 [0.70142872]
 [0.73931965]
 [0.65401481]]
Loss: 
 0.2536730156875411
+---------EPOCH 3000---------+
Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.0016318 ]
 [0.97901706]
 [0.97974103]
 [0.97376863]
 [0.97462981]
 [0.02907209]
 [0.03979911]]
Loss: 
 0.0006591798588608768
+---------EPOCH 6000---------+
Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]


In [8]:
predictions = np.rint(nn.feed_forward(X)).astype(int)

In [9]:
predictions

array([[0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0]])

In [10]:
accuracy_score(df['y'], predictions)

1.0

In [11]:
nn.weights1

array([[ 5.88792566,  6.65399437, -0.42273464, -3.83688057],
       [ 5.86758256, -3.55892031, -1.31324661,  6.56407698],
       [-0.24387017,  0.4884112 , -0.92872575,  0.95895643]])

In [12]:
nn.weights2

array([[14.8709841 ],
       [-9.23617042],
       [-3.2406653 ],
       [-9.83245911]])

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chock-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


### Parts
1. Gathering & Transforming the Data
2. Making MNIST a Binary Problem
3. Estimating your Neural Network (the part you focus on)

### Gathering the Data 

`keras` has a handy method to pull the mnist dataset for you. You'll notice that each observation is a 28x28 arrary which represents an image. Although most Neural Network frameworks can handle higher dimensional data, that is more overhead than necessary for us. We need to flatten the image to one long row which will be 784 values (28X28). Basically, you will be appending each row to one another to make on really long row. 

In [13]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [14]:
# input image dimensions
img_rows, img_cols = 28, 28

In [15]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [16]:
x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)

# Normalize Our Data
x_train = x_train / 255
x_test = x_test / 255

In [17]:
# Now the data should be in a format you're more familiar with
x_train.shape

(60000, 784)

### Making MNIST a Binary Problem 
MNIST is multiclass classification problem; however we haven't covered all the necessary techniques to handle this yet. You would need to one-hot encode the target, use a different loss metric, and use softmax activations for the last layer. This is all stuff we'll cover later this week, but let us simply the problem for now: Zero or all else.

In [18]:
y_temp = np.zeros(y_train.shape)
y_temp[np.where(y_train == 0.0)[0]] = 1
y_train_binary = y_temp

y_temp = np.zeros(y_test.shape)
y_temp[np.where(y_test == 0.0)[0]] = 1
y_test_binary = y_temp

In [19]:
# A Nice Binary target for ya to work with
y_train_binary

array([0., 1., 0., ..., 0., 0., 0.])

### Estimating Your `net

In [20]:
class NeuralNetwork2:
    def __init__(self, inputs=2, hiddenNodes=[3], outputNodes=1):
        # Set up Architecture of Neural Network
        self.inputs = inputs
        self.hiddenNodes = hiddenNodes
        self.outputNodes = outputNodes

        # Initial Weights
        if len(hiddenNodes) == 0:
            self.weights = [np.random.rand(self.inputs, self.outputNodes)]
        else:
            self.weights = [np.random.rand(self.inputs, self.hiddenNodes[0])]
            for i in range(1, len(hiddenNodes)):
                self.weights.append(np.random.rand((hiddenNodes[i-1]), hiddenNodes[i]))
            self.weights.append(np.random.rand(hiddenNodes[-1], outputNodes))
        
    def sigmoid(self, s):
        return 1 / (1 + np.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1 - s)
    
    def feed_forward(self, X):
        """
        Calculate the NN inference using feed forward.
        aka "predict"
        """
        
        # Weighted sum of inputs => hidden layer
        weighted_sum = np.dot(X, self.weights[0])
        self.activated_outputs = [self.sigmoid(weighted_sum)]
        for i in range(1, len(self.weights)):
            weighted_sum = np.dot(self.activated_outputs[i-1], self.weights[i])
            self.activated_outputs.append(self.sigmoid(weighted_sum))
                
        return self.activated_outputs[-1]
        
    def backward(self, X, y, o):
        """
        Backward propagate through the network
        """
        
        # Error in Output
        self.o_error = y - o
        
        # Apply Derivative of Sigmoid to error
        # How far off are we in relation to the Sigmoid f(x) of the output
        # ^- aka hidden => output
        self.o_delta = self.o_error * self.sigmoidPrime(o)
        z2_error = self.o_delta.dot(self.weights[-1].T)

        # Adjustment to final set of weights (hidden => output)
        if len(self.weights) == 1:
            self.weights[0] += X.T.dot(self.o_delta)
            return
        
        self.weights[-1] += self.activated_outputs[-2].T.dot(self.o_delta)
        
        # Hidden layer error
        for i in reversed(range(2, len(self.weights))):
            z2_delta = z2_error * self.sigmoidPrime(self.activated_outputs[i-1])
            z2_error = z2_delta.dot(self.weights[i-1].T)
        
            # Adjustment to interior set of weights (hidden => hidden)
            self.weights[i-1] += self.activated_outputs[i-2].T.dot(z2_delta)
        
        # Input layer error (if not same layer as output)
        z2_delta = z2_error * self.sigmoidPrime(self.activated_outputs[0])
        self.weights[0] += X.T.dot(z2_delta)

    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X, y, o)

    def fit(self, X, y, max_epochs, print_epochs):
        for i in range(max_epochs):
            if (i+ 1 in print_epochs): 
                print('+' + '---' * 3 + f'EPOCH {i + 1}' + '---' * 3 + '+')
                print('Input: \n', X)
                print('Actual Output: \n', y)
                print('Predicted Output: \n', str(self.feed_forward(X)))
                print("Loss: \n", str(np.mean(np.square(y - self.feed_forward(X)))))
            self.train(X, y)

In [21]:
nn2 = NeuralNetwork2(inputs=3, hiddenNodes=[4], outputNodes=1)

In [22]:
nn2.fit(X, y, 5000, [1, 2, 3, 5000])

+---------EPOCH 1---------+
Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.80257004]
 [0.86408408]
 [0.85686966]
 [0.84829118]
 [0.84042383]
 [0.89253992]
 [0.77624074]]
Loss: 
 0.30439077382104046
+---------EPOCH 2---------+
Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.73836496]
 [0.79894553]
 [0.79128119]
 [0.78425777]
 [0.77620421]
 [0.82947205]
 [0.71656786]]
Loss: 
 0.275327414811274
+---------EPOCH 3---------+
Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.66743346]
 [0.71869721]
 [0.71169972]
 [0.70700945]
 [0.699779  ]
 [0.74581564]
 [0.65147548]]
Loss: 
 0.2520504436634123
+---------EPOCH 5000---------+
Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 

In [23]:
predictions2 = np.rint(nn2.feed_forward(X))

In [24]:
accuracy_score(y, predictions2)

1.0

In [25]:
nn2 = NeuralNetwork2(inputs=3, hiddenNodes=[2, 3], outputNodes=1)

In [26]:
nn2.fit(X, y, 5000, [1, 2, 3, 5000])

+---------EPOCH 1---------+
Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.76939623]
 [0.77747185]
 [0.77460233]
 [0.76973203]
 [0.76518145]
 [0.78082521]
 [0.75827277]]
Loss: 
 0.2835888756899158
+---------EPOCH 2---------+
Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.70596606]
 [0.71238425]
 [0.71007668]
 [0.70624185]
 [0.70264958]
 [0.71506363]
 [0.69730101]]
Loss: 
 0.2624888933859016
+---------EPOCH 3---------+
Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.65111984]
 [0.65569921]
 [0.65403288]
 [0.65129278]
 [0.64872536]
 [0.65760742]
 [0.64497755]]
Loss: 
 0.2508039254311197
+---------EPOCH 5000---------+
Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 

In [27]:
predictions2 = np.rint(nn2.feed_forward(X))

In [28]:
accuracy_score(y, predictions2)

1.0

In [29]:
nn2 = NeuralNetwork2(inputs=3, hiddenNodes=[], outputNodes=1)

In [30]:
nn2.fit(X, y, 5000, [1, 2, 3, 5000])

+---------EPOCH 1---------+
Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.6168668 ]
 [0.71713106]
 [0.76676795]
 [0.61159092]
 [0.67125792]
 [0.8380995 ]
 [0.5       ]]
Loss: 
 0.24661148685100706
+---------EPOCH 2---------+
Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.57831235]
 [0.69120949]
 [0.73696675]
 [0.62009008]
 [0.67137555]
 [0.82056688]
 [0.5       ]]
Loss: 
 0.23923411983362658
+---------EPOCH 3---------+
Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.54262038]
 [0.66717056]
 [0.70847402]
 [0.62820394]
 [0.67196546]
 [0.80416053]
 [0.5       ]]
Loss: 
 0.23324469031218031
+---------EPOCH 5000---------+
Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0

In [31]:
predictions3 = np.rint(nn2.feed_forward(X))

In [32]:
accuracy_score(y, predictions3)

0.8571428571428571

In [33]:
nn_mnist = NeuralNetwork2(inputs=784, hiddenNodes=[3], outputNodes=1)

In [34]:
nn_mnist.fit(x_train[:10], np.expand_dims(y_train_binary[:10], axis=1), 1000, [1, 2, 3, 1000])

+---------EPOCH 1---------+
Input: 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Actual Output: 
 [[0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Predicted Output: 
 [[0.69486119]
 [0.69486119]
 [0.69486119]
 [0.69486119]
 [0.69486119]
 [0.69486119]
 [0.69486119]
 [0.69486119]
 [0.69486119]
 [0.69486119]]
Loss: 
 0.4438598345322967
+---------EPOCH 2---------+
Input: 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Actual Output: 
 [[0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Predicted Output: 
 [[0.04922427]
 [0.04922427]
 [0.04922427]
 [0.04922427]
 [0.04922427]
 [0.04922427]
 [0.04922427]
 [0.04922427]
 [0.04922427]
 [0.04922427]]
Loss: 
 0.09257817502628944
+---------EPOCH 3---------+
Input: 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0

In [35]:
mnist_predict = nn_mnist.feed_forward(x_train[:10])

In [36]:
accuracy_score(y_train_binary[:10], np.rint(mnist_predict))

0.9

In [37]:
np.bincount(y_train_binary[:10].astype(int))

array([9, 1], dtype=int64)

In [38]:
np.bincount(np.rint(mnist_predict[0]).astype(int))

array([1], dtype=int64)

In [39]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt

In [40]:
model = keras.Sequential([
    keras.layers.Dense(8, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

In [41]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [42]:
model.fit(x_train, y_train, epochs=10)

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 2s 35us/sample - loss: 0.5536 - accuracy: 0.8413
Epoch 2/10
60000/60000 [==============================] - 2s 29us/sample - loss: 0.3236 - accuracy: 0.9083
Epoch 3/10
60000/60000 [==============================] - 2s 28us/sample - loss: 0.3002 - accuracy: 0.9145
Epoch 4/10
60000/60000 [==============================] - 2s 27us/sample - loss: 0.2891 - accuracy: 0.9179
Epoch 5/10
60000/60000 [==============================] - 2s 26us/sample - loss: 0.2815 - accuracy: 0.9203
Epoch 6/10
60000/60000 [==============================] - 2s 26us/sample - loss: 0.2757 - accuracy: 0.9220
Epoch 7/10
60000/60000 [==============================] - 2s 26us/sample - loss: 0.2710 - accuracy: 0.9231
Epoch 8/10
60000/60000 [==============================] - 2s 27us/sample - loss: 0.2673 - accuracy: 0.9243
Epoch 9/10
60000/60000 [==============================] - 2s 33us/sample - loss: 0.2643 - accuracy: 0.9248
Epoch 10/10
60

In [43]:
tf_predictions = model.predict(x_test)

In [47]:
accuracy_score(y_test, np.argmax(tf_predictions, axis=1))

0.9224

## Stretch Goals: 

- Make MNIST a multiclass problem using cross entropy & soft-max
- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?